### Projects of using Spark
#### 16300200020 Yanjian Zhang

Use the linux command to delete the first 76 lines and redirect to the data_new.txt file.Then delete the last 49 lines and redirect to the data_new.txt. We can see that the total numbers of lines are 49611709

Introduce the spark package and configure the spark context.

In [1]:
import os
os.getcwd()

'/home/bigdatalab06'

In [2]:
from __future__ import print_function,division
from pyspark import SparkConf,SparkContext
from pyspark.sql import SparkSession
from itertools import islice
import re
conf = SparkConf().setAppName("turkey").setMaster("local").set("spark.executor.memory","12g")
conf.set("spark.executor.cores","1").set("spark.memory.offHeap.enabled","true")
conf.set("spark.executor.memoryOverhead","4096").set("spark.memory.offHeap.size","4096")
sc = SparkContext(conf=conf)


In [3]:
from pyspark.ml.linalg import Vector,Vectors
from pyspark.sql import Row
from pyspark.ml import Pipeline
from pyspark.ml.feature import IndexToString,StringIndexer,VectorIndexer
from pyspark import SparkConf,SparkContext
from pyspark.sql import SparkSession
import pyspark.sql as sql

data = sc.textFile('hdfs://localhost:9001/user/data_200w.txt').map(lambda x : re.split("\t", x))
# classOfFirstName = data.map(lambda x: x[2]).distinct().persist()
# print(classOfFirstName.take(100))

In [4]:
# print(len(classOfFirstName))

In [5]:
# classOfLastName = data.map(lambda x: x[3]).distinct().persist()

In [6]:
from pyspark.sql import Row
spark = SparkSession(sc)
from pyspark.ml.classification import NaiveBayes
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoderEstimator



In [7]:
from pyspark.ml.feature import OneHotEncoderEstimator
currentdata = data.map(lambda x: [x[3], x[4][0:2], x[5][0:2]])
print(currentdata.take(10))

# print(data.map(lambda x: x[14]).distinct().count()) # 207482
# print(data.map(lambda x: x[14].split()[0]).distinct().count()) # 119346
schema = ['last_name','mother_first','father_first']
df = spark.createDataFrame(currentdata, schema)

indexer = [StringIndexer(inputCol = column, outputCol = column + "_index") for column in schema]
# onehotindexer = OneHotEncoderEstimator(inputCols = [column + "_index" for column in schema], outputCols = [column + "_onehot" for column in schema])
pipeline = Pipeline(stages = indexer)
df = pipeline.fit(df).transform(df)
df = df.drop(*schema)
df.show()


[['ZENGIN', 'ZE', 'OS'], ['YILDIRIM', 'ZO', 'IS'], ['CETIN', 'ES', 'HA'], ['GENC', 'EL', 'IS'], ['YILDIRIM', 'SE', 'MU'], ['YILDIRIM', 'SE', 'MU'], ['GUZEL', 'KA', 'OS'], ['DURMAZ', 'YE', 'SU'], ['ZENGIN', 'MA', 'ME'], ['AKBULUT', 'SA', 'IH']]
+---------------+------------------+------------------+
|last_name_index|mother_first_index|father_first_index|
+---------------+------------------+------------------+
|          119.0|               7.0|              13.0|
|            8.0|              71.0|               9.0|
|           21.0|              27.0|               1.0|
|           46.0|              16.0|               9.0|
|            8.0|               4.0|               2.0|
|            8.0|               4.0|               2.0|
|          147.0|              21.0|              13.0|
|          207.0|              38.0|              11.0|
|          119.0|              23.0|               0.0|
|           73.0|               6.0|              52.0|
|         4713.0|           

In [8]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import LogisticRegression
vectorForm = df.rdd.map(lambda row: (row[0], Vectors.dense(row[1:3])))
Vectordf = spark.createDataFrame(vectorForm, ['label', 'features'])
trainData, validData, testData = Vectordf.rdd.randomSplit([0.7,0.1,0.2])
# linear regression
# lr = LogisticRegression(regParam=0.1, elasticNetParam=1.0, family="multinomial")
# lrmodel = lr.fit(trainData.toDF())
# predictions = lrmodel.transform(testData.toDF())
# randomforest
# rf = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=2)
# rfmodel = rf.fit(trainData.toDF())
# predictions = rfmodel.transform(testData.toDF())
# bayes
nb = NaiveBayes(labelCol="label", featuresCol="features")
nbmodel = nb.fit(trainData.toDF())
predictions = nbmodel.transform(testData.toDF())
p1 = predictions.select('probability', 'label')

In [9]:
p1 = predictions.select('probability', 'label')

In [10]:
p1.show(30)

+--------------------+-------+
|         probability|  label|
+--------------------+-------+
|[3.32396055889341...|   21.0|
|[0.01477152882879...|    8.0|
|[0.01477152882879...|    8.0|
|[5.63458137170562...|  207.0|
|[0.01841569831168...|  119.0|
|[0.00454672820864...|  249.0|
|[4.31567137653907...|  119.0|
|[3.26313555866030...| 1340.0|
|[0.01522904020725...|  147.0|
|[0.01411537293969...| 1340.0|
|[3.06185122435452...|  119.0|
|[0.00121803150355...| 1789.0|
|[0.02463701494476...|  655.0|
|[5.03637807370170...|  282.0|
|[0.00809547757635...|  852.0|
|[0.00299050192203...|    1.0|
|[0.01636529538344...|   41.0|
|[3.26313555866030...|  655.0|
|[7.40897083751392...|  655.0|
|[0.01011708266086...|25695.0|
|[0.02078637052082...|   12.0|
|[0.00111837008220...|    2.0|
|[2.42943512357890...|  852.0|
|[0.00793810542262...|   38.0|
|[0.01892456334895...|   74.0|
|[0.01765684005535...|  162.0|
|[0.02017383096703...|  655.0|
|[0.01127719815823...|   85.0|
|[0.01607711684215...|   85.0|
|[0.0095

In [ ]:
# print(p1.rdd.map(lambda x: (list(enumerate(x[0])), x[1])).take(10))
hits = p1.rdd.map(lambda x: (sorted(list(enumerate(x[0]))[0:5], key = lambda y: -y[1]), x[1])).map(lambda row: ([x[0] for x in row[0]], row[1])) # 按index 排序
# hits = hits.map(lambda x: [ 1 if x[0][i]== x[1] else 0 for i in range(5)])
total = hits.count()
hits1 = hits.map(lambda x:  1 if x[0][0]== x[1] else 0).filter(lambda x: x == 1).count()/total
print(hits1)
hits2 = hits.map(lambda x:  1 if x[0][1]== x[1] else 0).filter(lambda x: x == 1).count()/total
print(hits2)
hits3 = hits.map(lambda x:  1 if x[0][2]== x[1] else 0).filter(lambda x: x == 1).count()/total
print(hits3)
hits4 = hits.map(lambda x:  1 if x[0][3]== x[1] else 0).filter(lambda x: x == 1).count()/total
print(hits4)
hits5 = hits.map(lambda x:  1 if x[0][4]== x[1] else 0).filter(lambda x: x == 1).count()/total
print(hits5)
# hits.take(2000)

In [ ]:
# topn = [p1.rdd.map(lambda x: (hit_n(x[0],1),x[1])).filter(lambda x: int(x[1]) in x[0]).count()/float(top1_pre.count()) for i]

In [ ]:
# print(top1_pre.take(10))
# print(top1_pre.filter(lambda x: int(x[1]) in x[0]).count()/float(top1_pre.count()))